In [1]:
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
import pymysql

In [2]:
#DB 연결
def db_conn(host='db-dfmba.cnm8u4m2cbtp.ap-northeast-2.rds.amazonaws.com' , port=3306):
    engine = create_engine(f'mysql+pymysql://root:alexalex@{host}:{port}/stock_db')
    con = pymysql.connect(user = 'root', 
                        passwd ='alexalex',
                        host = f'{host}', 
                        db='stock_db', 
                        charset = 'utf8',
                        cursorclass = pymysql.cursors.DictCursor)
    mycursor = con.cursor()
    return engine, con, mycursor

engine, con, mycursor = db_conn()

# DB에서 주식 리스트 가저오기 
sql = "select * from krx_list"
mycursor.execute(sql)
result = mycursor.fetchall()
# con.close()
df_list = pd.DataFrame(result)

In [3]:
df_list = df_list.loc[df_list['Market'] != 'KONEX']

In [4]:
df_list

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,73500,3,0,0.00,74500,74700,73500,26198776,1935963757832,438779017425000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,189200,2,-6500,-3.32,194400,195200,188200,5958829,1131954708965,137738047458000,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,331000,1,3000,0.91,331500,335500,328000,264516,87510775500,77454000000000,234000000,STK
3,005380,KR7005380001,현대차,KOSPI,,253000,2,-4000,-1.56,259500,260000,253000,1014549,258540736466,52982296323000,209416191,STK
4,207940,KR7207940008,삼성바이오로직스,KOSPI,,730000,2,-2000,-0.27,739000,740000,728000,51389,37599700000,51957020000000,71174000,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,032685,KR7032681009,소프트센우,KOSDAQ,중견기업부,9540,1,80,0.85,9400,9600,9400,462,4401350,3407134680,357142,KSQ
2800,001525,KR7001521004,동양우,KOSPI,,5490,1,40,0.73,5450,5490,5370,1026,5553660,3391343190,617731,STK
2801,021045,KR7021041009,대호특수강우,KOSDAQ,중견기업부,7950,2,-50,-0.63,7850,7960,7850,115,903060,3372755700,424246,KSQ
2803,001527,KR7001522002,동양2우B,KOSPI,,10810,2,-280,-2.52,11080,11080,10810,133,1453730,3335630890,308569,STK


In [7]:
engine, con, mycursor = db_conn()

# 테이블에서 마지막 데이터 일자 가져오기 
sql = "select max(date) as max_date from kr_stock_price"

try:
    mycursor.execute(sql)
    result = mycursor.fetchone()
    print(f'Max date is {result}')
    if result['max_date'] is not None:
        max_date = pd.to_datetime(result['max_date'])
    else:
        max_date = datetime.datetime.today() - timedelta(days=6*365)
except Exception as e:
    print(f'Exception : {e}')

con.close()

Max date is {'max_date': datetime.datetime(2025, 1, 3, 0, 0)}


In [9]:

# 오늘 날짜를 기준으로 3년 전 날짜 계산
end_date = datetime.today()
start_date = max_date + timedelta(days=1)

# 종목 코드, 예를 들어 삼성전자(005930)
# stock_code = '005930'

print(f'Start date is {start_date} and End date is {end_date}')

engine, con, mycursor = db_conn()

count = 0
for stock_code in df_list.iloc[:,0] : 
    print(f'\r {stock_code} {count} / {len(df_list.iloc[:,0])}')
    df = fdr.DataReader(stock_code, start=start_date, end=end_date)
    df['Code'] = stock_code
    df.reset_index(inplace=True)
    # df.set_index(['Code', 'Date'], inplace=True)
    col1= df.columns[-1:].to_list()
    col2= df.columns[:-1].to_list()
    new_col = col1+col2
    df = df[new_col]
    # print(df)
    count = count + 1

    df.to_sql(name = 'kr_stock_price', con=engine, if_exists='append', index=False)


Start date is 2025-01-04 00:00:00 and End date is 2025-01-06 11:16:48.832554
 024070 0 / 2685
 076610 1 / 2685
 045300 2 / 2685
 246250 3 / 2685
 009140 4 / 2685
 365900 5 / 2685
 005320 6 / 2685
 190650 7 / 2685
 191410 8 / 2685
 012620 9 / 2685
 115480 10 / 2685
 002690 11 / 2685
 104040 12 / 2685
 093380 13 / 2685
 039310 14 / 2685
 002820 15 / 2685
 094860 16 / 2685
 290560 17 / 2685
 002410 18 / 2685
 208350 19 / 2685
 011090 20 / 2685
 133820 21 / 2685
 439250 22 / 2685
 001070 23 / 2685
 060900 24 / 2685
 380540 25 / 2685
 025620 26 / 2685
 291810 27 / 2685
 000950 28 / 2685
 123010 29 / 2685
 099410 30 / 2685
 008470 31 / 2685
 196490 32 / 2685
 179530 33 / 2685
 083640 34 / 2685
 290720 35 / 2685
 192390 36 / 2685
 018620 37 / 2685
 001420 38 / 2685
 086060 39 / 2685
 333050 40 / 2685
 091340 41 / 2685
 001140 42 / 2685
 017000 43 / 2685
 424760 44 / 2685
 014910 45 / 2685
 177350 46 / 2685
 148250 47 / 2685
 003535 48 / 2685
 084870 49 / 2685
 010400 50 / 2685
 001465 51 / 26